In [1]:
! pip install torch

In [1]:
from typing import Tuple

import torch
from bastionai.client import connect
from bastionai.pb.remote_torch_pb2 import TestConfig, TrainConfig
from bastionai.psg.nn import Linear
from torch import Tensor
from torch.nn import Module
from torch.utils.data import Dataset


Below is a simple neural network that simply employs a Linear layer to perform linear regression.

In [2]:
class LReg(Module):
    def __init__(self) -> None:
        super().__init__()
        self.fc1 = Linear(1, 1, 2)

    def forward(self, x: Tensor) -> Tensor:
        return self.fc1(x)

A simple dataloader described below serving two tensors both holding 4 elements.

In [3]:
class LRegDataset(Dataset):
    def __init__(self) -> None:
        super().__init__()
        self.X = torch.tensor([[0.0], [1.0], [0.5], [0.2]])
        self.Y = torch.tensor([[0.0], [2.0], [1.0], [0.4]])

    def __len__(self) -> int:
        return 4

    def __getitem__(self, idx: int) -> Tuple[torch.Tensor, torch.Tensor]:
        return (self.X[idx], self.Y[idx])

Initializes both the model and the dataloader.

In [4]:
lreg_model = LReg()
lreg_dataset = LRegDataset()

Once BastionAI receives artifacts (both datasets and models,) it returns references.

In [6]:
client = connect(addr="localhost", port=50053)

model_ref = client.send_model(
    lreg_model, "1D Linear Regression Model", b"secret")
print(f"Model ref: {model_ref}")

dataset_ref = client.send_dataset(
    lreg_dataset, "Dummy 1D Linear Regression Dataset (param is 2)", b'secret')
print(f"Dataset ref: {dataset_ref}")

Model ref: identifier: "57bae2ed-bd2a-495e-8321-c0dbd43220db"
description: "1D Linear Regression Model"

Dataset ref: identifier: "9af94af1-296f-4360-86e2-e2095e3ba532"
description: "Dummy 1D Linear Regression Dataset (param is 2)"



Below, we fetch all the available models and datasets on BastionAI.

In [8]:
client = connect("localhost", 50053)
available_models = client.get_available_models()
print(available_models)

available_datasets = client.get_available_datasets()
print(available_datasets)


list {
  identifier: "57bae2ed-bd2a-495e-8321-c0dbd43220db"
  description: "1D Linear Regression Model"
}

list {
  identifier: "9af94af1-296f-4360-86e2-e2095e3ba532"
  description: "Dummy 1D Linear Regression Dataset (param is 2)"
}



Models on BastionAI can be either trained on GPU or CPU. Below, we use the `get_devices` endpoint to see the available devices on BastionAI.

In [10]:
client = connect("localhost", 50053)

available_devices = client.get_available_devices()
print(available_devices)


['cpu']


To test the power of BastionAI, we train the simple Linear regression neural network defined above.

Create training configurations.

In [17]:
from bastionai.utils.utils import create_training_config, Optimizers


In [18]:
client = connect(addr="localhost", port=50053)
training_config = create_training_config(model_ref,
                                         dataset_ref,
                                         batch_size=2,
                                         epochs=100,
                                         learning_rate=0.1,
                                         weight_decay=0.,
                                         noise_multiplier=0.1,
                                         max_grad_norm=1.,
                                         extra_args={
                                             "momentum": 0.,
                                             "dampening": 0.,
                                             "nesterov": False
                                         },
                                         optimizer=Optimizers.SGD)

client.train(training_config)


Epoch 100/100 - train: 100%|████████████████████| 2/2 [00:00<00:00, 659.43batch/s, loss (l2)=0.0236] 


We securely fetch the trained weights from BastionAI.

In [ ]:
client = connect("localhost", 50051)
client.fetch_model_weights(lreg_model, model_ref)
print(f"Weight: {lreg_model.fc1.inner.expanded_weight}")
